In [1]:
import geopandas as gpd
import os

# === BASE DIRECTORY ===
base_dir = r"C:\Users\natda\OneDrive - Northeastern University\Desktop\NatDave\Academics\PhD_NU\RESEARCH\Traffic_Stress\Boston"

# === FILE PATHS ===
roads_path = os.path.join(base_dir, "street_network.shp")

# === LOAD SHAPEFILE ===
roads = gpd.read_file(roads_path)

# === FILTER ONE-WAY STREETS ===
one_way_one_lane_roads = roads[(roads['StOperNEU'] == 1) & (roads['qDirLanes2'] < 2)]

In [2]:
# === FUNCTION TO IDENTIFY CONNECTED SEGMENTS BASED ON STREET NAME ===
def identify_connected_segments(road_gdf):
    """Identify connected road segments by street name, ensuring segments are connected by touching."""
    connected_segments = []  # To store connected components
    
    for street_name in road_gdf['STREETNAME'].unique():
        street_segments = road_gdf[road_gdf['STREETNAME'] == street_name]
        
        # Initialize groupings for connected segments
        group = []
        
        for _, row in street_segments.iterrows():
            road_geom = row['geometry']
            connected = False
            
            # Check if this segment is already connected to any in the group
            for seg in group:
                if road_geom.intersects(seg['geometry']):
                    connected = True
                    break
            
            # If connected, add it to the group, otherwise start a new group
            if connected:
                group.append(row)
            else:
                # Save the current group and start a new one
                if group:
                    connected_segments.append(group)
                group = [row]
        
        # Append the last group
        if group:
            connected_segments.append(group)
    
    return connected_segments

# === APPLY FUNCTION TO IDENTIFY CONNECTED SEGMENTS BY STREET NAME ===
connected_segments = identify_connected_segments(one_way_one_lane_roads)

# === FILTER BY LENGTH > 600m ===
long_streets = []

for group in connected_segments:
    total_length = sum([seg['geometry'].length for seg in group])
    if total_length > 600:  # Only keep streets with length > 600m
        long_streets.append(group)

# === OUTPUT THE RESULTS ===
print(f"Identified {len(long_streets)} connected one-way street neighborhoods with total length greater than 600m.")

# Create GeoDataFrame for long streets (just for visualization)
long_streets_gdf = gpd.GeoDataFrame([seg for group in long_streets for seg in group])

# Set CRS to match the original roads GeoDataFrame
long_streets_gdf.set_crs(roads.crs, inplace=True)

# Save to file (for visualization)
long_streets_path = os.path.join(base_dir, "long_streets.shp")
long_streets_gdf.to_file(long_streets_path)

# === UPDATE ADT_2025 FOR THE ORIGINAL ROADS (INCLUDED IN one_way_one_lane_ROADS) ===
# Now, we use the 'unique_id' to match the long street segments and update ADT_2025
# Update ADT_2025 for the segments in long streets (999) and the others (599)

# Get unique_ids for long street segments
long_streets_unique_ids = [seg['unique_id'] for group in long_streets for seg in group]

# Update ADT_2025 for the segments that are part of long streets (999)
roads.loc[roads['unique_id'].isin(long_streets_unique_ids), 'ADT_2025'] = 999

# Update ADT_2025 for the rest of the one-way, one-lane roads (599)
roads.loc[roads['unique_id'].isin(one_way_one_lane_roads['unique_id']) & 
          ~roads['unique_id'].isin(long_streets_unique_ids), 'ADT_2025'] = 599

Identified 58 connected one-way street neighborhoods with total length greater than 600m.


In [3]:
roads.loc[roads['ADT_2025'] == 8001, 'ADT_2025'] = 12001

In [4]:
roads['ADT_2025'].value_counts()

ADT_2025
3001     16088
749      10471
599       4280
0         2224
12001     1809
751        740
2999       456
999        450
1501       156
1499        66
7999        56
Name: count, dtype: int64

In [ ]:
# === SAVE THE UPDATED ROADS SHAPEFILE ===
roads.to_file(roads_path, driver="ESRI Shapefile")